## Let's answer some questions and visualise some data

### By using pandas and plotly

In [1]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connecting to the WizardsCo database
connection = sqlite3.connect('wizards_co.db')
cursor = connection.cursor()

### What is the number of installations that the company is doing every month?

In [3]:
monthly_installations_query = """SELECT COUNT(*) AS number_of_installations, installation_month
                FROM fact_installation
                GROUP BY installation_month
                ORDER BY installation_month;"""

monthly_installations = pd.read_sql_query(monthly_installations_query, connection)

monthly_installations

,number_of_installations,installation_month
0,2,04
1,2,05
2,1,06
3,5,07
4,8,08
5,8,09
6,10,10


In [4]:
fig1 = px.bar(monthly_installations, x='installation_month', y='number_of_installations', text_auto=True)
fig1.write_image("images/installations_per_month.jpeg")
fig1.show()

### Which product category brings us more revenues?

In [5]:
revenues_per_category_query = """SELECT SUM(price) AS revenues, product_category
                            FROM fact_installation
                            GROUP BY product_category
                            ORDER BY revenues DESC"""

revenues_per_category = pd.read_sql_query(revenues_per_category_query, connection)

revenues_per_category

,revenues,product_category
0,111105.0,Dangerous Items
1,7101.0,IT & Network Stuff
2,570.0,Beauty Accessories
3,36.0,Miscellaneous
4,12.0,Medical Device


In [6]:
fig2 = px.bar(revenues_per_category, x='product_category', y='revenues', text_auto=True)
fig2.write_image("images/revenues_per_category.jpeg")
fig2.show()

### Which region of the world is our best market?

In [7]:
revenues_per_region_query = """SELECT SUM(price) AS revenues, region
                            FROM fact_installation
                            GROUP BY region
                            ORDER BY revenues DESC"""

revenues_per_region = pd.read_sql_query(revenues_per_region_query, connection)

revenues_per_region

,revenues,region
0,44703.0,Europe
1,37071.0,America
2,37050.0,Asia


In [8]:
fig3 = px.bar(revenues_per_region, x='region', y='revenues', text_auto=True)
fig3.write_image("images/revenues_per_region.jpeg")
fig3.show()